In [1]:
import pandas as pd
import xgboost as xgb
from xgboost import XGBRegressor, plot_importance
from colorama import Fore, Back, Style
import os
import numpy as np
import datetime as dt
import time
import warnings
warnings.filterwarnings("ignore")

In [2]:
package_dir = os.path.dirname(os.path.abspath('FICHIER DU 09 BOOST SAVE'))
LeChemin = '/'+package_dir+'/20191211 BACKTEST/'
#LeChemin = ''
list_dir = os.listdir(LeChemin+'DL_2019-12-06/.')


In [3]:
LaBase = pd.read_csv(LeChemin+'LaBase_10.csv')
try:
    LaBAse = LaBase.drop(['Unnamed: 0'],axis=1)
except :
    pass
LaBase.index = pd.to_datetime(LaBase['Date'],format='%Y-%m-%d')
LaBase = LaBase.drop(['Date'],axis=1)
LaBase = LaBase.drop(['Close'],axis=1)
LaBase['Close'] = LaBase['Adj Close']
LaBase = LaBase.drop(['Adj Close'],axis=1)

In [4]:
NEW_LIST = []
TIME_LIST = []
for pikunichihouahoua in range(0,len(str(list_dir).split(','))):
    if str(list_dir).split(',')[pikunichihouahoua].split(',')[0].split('.')[1].split("'")[0]=='yaml':
        NEW_LIST.append(str(list_dir).split(',')[pikunichihouahoua].split(',')[0].split('.')[0].split("'")[1].split('_')[1])
NEW_LIST = sorted(set(NEW_LIST))

TIME_LIST = sorted(set(LaBase.index.strftime('%Y-%m-%d')))

In [5]:
%%time
tmps55=time.time()
print(Fore.YELLOW,'ON GENERE LES BOOST',Style.RESET_ALL)


for symbol in NEW_LIST:
    CLOSE_S = []
    SYMBOL = []
    PRED = []
    MOMENTUM = []
    df1 = pd.DataFrame()
    i = 0
    for momentum in TIME_LIST[800:]:
        
        long = len(NEW_LIST[800:])
        
        i+=1
        if i == 1:
            print('\r',Fore.GREEN,' |==',Fore.BLUE,'O',Fore.RED,'==================|',end='',flush=True)
        if i == round(1 * long / 9):
            print('\r',Fore.GREEN,' |====',Fore.BLUE,'O',Fore.RED,'================|',end='',flush=True)
        if i == round(2 * long / 9):
            print('\r',Fore.GREEN,' |======',Fore.BLUE,'O',Fore.RED,'==============|',end='',flush=True)
        if i == round(3 * long / 9):
            print('\r',Fore.GREEN,' |========',Fore.BLUE,'O',Fore.RED,'============|',end='',flush=True)
        if i == round(4 * long / 9):
            print('\r',Fore.GREEN,' |==========',Fore.BLUE,'O',Fore.RED,'==========|',end='',flush=True)
        if i == round(5 * long / 9):
            print('\r',Fore.GREEN,' |============',Fore.BLUE,'O',Fore.RED,'========|',end='',flush=True)
        if i == round(6 * long / 9):
            print('\r',Fore.GREEN,' |==============',Fore.BLUE,'O',Fore.RED,'======|',end='',flush=True)
        if i == round(7 * long / 9):
            print('\r',Fore.GREEN,' |================',Fore.BLUE,'O',Fore.RED,'====|',end='',flush=True)
        if i == round(8 * long / 9):
            print('\r',Fore.GREEN,' |==================',Fore.BLUE,'O',Fore.RED,'==|',Style.RESET_ALL,end='',flush=True)
        
        df = LaBase[LaBase['Symbol'] == symbol]
        df = df.drop(['Symbol'],axis=1)
        df = df[df.index<momentum]
        seed = 770
        np.random.seed(seed)
        X = df.copy()
        X = X.drop(['Close'],axis=1)
        X['Close'] = df['Close']
        y = X.iloc[:,-1]
        Xtrain = X.iloc[:-2,:-1]
        Xtest = X.iloc[-2:-1,:-1]
        yshift = y.shift(-1)
        ytrain = yshift.iloc[:-2]
        ytest = yshift.iloc[-2:-1]

        model = xgb.XGBRegressor(n_estimators=20000, learning_rate=1, gamma=1, subsample=1, colsample_bytree=1, max_depth=100,objective='reg:squarederror')

        model.fit( Xtrain, ytrain, early_stopping_rounds=150, eval_set=[(Xtest, ytest)], verbose=0)

        ytrain_pred = model.predict(Xtrain)

        y_pred = model.predict(Xtest)

        pred = model.predict(df[df.index==momentum].iloc[:,:-1]) #X.iloc[:,:-1])

        try:
            PRED.append(pred[0])
        except:
            PRED.append(9999)
        try:
            SYMBOL.append(symbol)
        except:
            SYMBOL.append('OOPS!')
        try:
            CLOSE_S.append(df.iloc[-1]['Close'])
        except:
            CLOSE_S.append(9999)
        try:
            MOMENTUM.append(momentum)
        except:
            MOMENTUM.append(9999)
        #df = df.dropna()
    df1['Date'] = MOMENTUM
    df1['Ticker'] = SYMBOL
    df1['Vrai Close du Lendemain'] = CLOSE_S
    df1['Vrai Close du Lendemain'] = df1['Vrai Close du Lendemain'].shift(-1)
    df1['Close.Pred'] = PRED
    df1['%'] = (df1['Vrai Close du Lendemain']-df1['Close.Pred'])*100/df1['Vrai Close du Lendemain']
    df1.to_csv(LeChemin+'TEST_BOOST/'+symbol+'.csv')
tmps22=round(time.time()-tmps55,2)
print(Fore.BLUE,"Time for completing Generation = %f" %tmps22,'seconds \n',Style.RESET_ALL)

 ON GENERE LES BOOST 
   |==  O  ==================|

KeyboardInterrupt: 

In [17]:
len(PRED),len(df['Close'].shift(-1)),len(SYMBOL),i

(328, 1011, 328, 329)

211

In [24]:
df.iloc[-1]['Close'].shift(-1)

AttributeError: 'numpy.float64' object has no attribute 'shift'